## KoBERT 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 1.8 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=05d74f78c074bd40309d8785a672b4fe936a199d77a719ef2064c241316dc2bc
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 769 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 51.2 MB/s 
     |████████████████████████████████| 3.0 MB 34.2 MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-jxg9ba4t
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-jxg9ba4t
     |████████████████████████████████| 132 kB 4.3 MB/s 
     |████████████████████████████████| 4.5 MB 40.3 MB/s 
     |████████████████████████████████| 3.8 MB 31.2 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 6.5 MB 31.9 MB/s 
     |████████████████████████████████| 79 kB 4.5 MB/s 
     |████████████████████████████████| 8.6 MB 10.2 MB/s 
     |████████████████████████████████| 127 kB 36.8 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=10ab2193deb09188cc54ea819e56ef787d0a042ec624f000c4890f6404c7c88d
  Stored in directory: /tmp/pip-ephem-wheel-cache-u7lfsjs2/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd05

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


## 데이터 가져오기

In [ ]:
import numpy as np
import pandas as pd
import os

path = '/content/drive/Shareddrives/NLP모델링/강의평 데이터/크롤링 통합'
file_list = os.listdir(path)

In [ ]:
df = pd.DataFrame()
for i in file_list:
    data = pd.read_csv(path+ '/' + i)
    df = pd.concat([df,data])
    
review = df.reset_index(drop = True)

In [ ]:
review_dropd = review.drop_duplicates(keep = 'first')

In [ ]:
review_dropd.head()

,학정번호,강의명,교수명,유의사항,별점,수강시기,강의평
0,YCC1001-01-00,대학기본영어Ⅰ,전진희,"대면강의, 절대평가",4.0,19년 1학기,과제는 자잘하게 많이 내주시지만 큰 시간이 소요된다는 생각을 해본적은 없음. 수업 ...
1,YCC1001-01-00,대학기본영어Ⅰ,전진희,"대면강의, 절대평가",3.0,19년 1학기,과제가 진짜 일주일에 두번씩 있음\n어려운건 아닌데 나처럼 귀찮아하다가 한두개 안해...
2,YCC1001-01-00,대학기본영어Ⅰ,전진희,"대면강의, 절대평가",3.0,19년 1학기,잔 과제가 많고 쪽지 시험도 많다. 그리고 되게 꼼꼼하셔서 그냥 숙제로 해 오는 받...
3,YCC1001-01-00,대학기본영어Ⅰ,전진희,"대면강의, 절대평가",5.0,19년 1학기,프롶 진희전 . 처음엔 되게 깐깐할 것 같았는데 나름의 인간적 면모도 보여주시고 좋...
4,YCC1001-01-00,대학기본영어Ⅰ,전진희,"대면강의, 절대평가",3.0,19년 1학기,수업은 간단한데 과제가 매우 많음 과제는 쉬운편\n시험이랑 퀴즈는 교재 외우면 됨\...


In [ ]:
review_dropd['별점'] = review_dropd['별점'].apply(lambda x : int(x-1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data_list = []
for q, label in zip(review_dropd['강의평'], review_dropd['별점'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[-1])

['과제는 자잘하게 많이 내주시지만 큰 시간이 소요된다는 생각을 해본적은 없음. 수업 때 학생들을 굉장히 최우선적으로 생각해주신다는 것이 느껴짐. 학점은 한만큼 나옴.', '3']
['걍 최고니 무조건 들으시길 바랍니다\n시험도 없고 발표 한번만 하면 에이쁠 나옵니다\n게다가 제가 아프고 사정있고 해서 3번이나 결석했는데도\n에이쁠 주셨습니다\n류부열 교수님 사랑합니다', '4']


## Before Train


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.10, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

29674
3298


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[0]

#첫 번째는 패딩된 시퀀스, 두 번째는 길이와 타입에 대한 내용, 세 번째는 어텐션 마스크 시퀀스

(array([   2, 3005, 7086, 3432, 6412, 7096, 6891, 4297, 6942, 7203, 1434,
        6733, 5782, 5760, 3219, 6116, 1800, 1042, 6116,  517, 5626, 4926,
         517, 5348, 6701,  517,   54, 1221, 5850, 3005, 2129, 2355, 7160,
        1042, 7836, 4920, 3275,  889, 1342, 5793, 6827, 6701,  517,   54,
        3005, 3942, 2358,  517, 6751, 7074, 6733, 2471, 1189, 2129,  517,
        5618, 5618, 7996, 3824, 7074, 6586,  517,   54,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Train

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    torch.save(model, '/content/drive/Shareddrives/NLP모델링/상윤/{}0405.h5'.format(batch_id))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/464 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.8276516199111938 train acc 0.125
epoch 1 batch id 201 loss 1.2675527334213257 train acc 0.44426305970149255
epoch 1 batch id 401 loss 0.9391472935676575 train acc 0.5049096009975063
epoch 1 train acc 0.515625


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 14 test acc 0.6579821832579186


  0%|          | 0/464 [00:00<?, ?it/s]

epoch 15 batch id 1 loss 0.015503463335335255 train acc 1.0
epoch 15 batch id 201 loss 0.2921636998653412 train acc 0.9852300995024875
epoch 15 batch id 401 loss 0.07233614474534988 train acc 0.9851543017456359
epoch 15 train acc 0.9858903556034483


  0%|          | 0/52 [00:00<?, ?it/s]

epoch 15 test acc 0.6594845871040724


## Predict

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("1점")
            elif np.argmax(logits) == 1:
                test_eval.append("2점")
            elif np.argmax(logits) == 2:
                test_eval.append("3점")
            elif np.argmax(logits) == 3:
                test_eval.append("4점")
            elif np.argmax(logits) == 4:
                test_eval.append("5점")

        print(">> 입력하신 내용으로 짐작하기에 이 강의는 " + test_eval[0] + "짜리 강의로 판단됩니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
end = 1
print("강의평 예측기를 작동합니다.")
while end == 1 :
    sentence = input("문장을 입력해주세요 : ")
    if sentence == '종료' :
        break
    predict(sentence)
    print("\n")
print("강의평 예측기를 종료합니다.")

강의평 예측기를 작동합니다.
문장을 입력해주세요 : 교수님과 학생 간에 커뮤니케이션이 거의 없습니다. 과제는 엄청 어렵고 시험도 어려운데 이걸 수업에서 대비시켜 주지 않습니다. 각자 알아서 공부해야해요. 학점은 적당히 주십니다


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용으로 짐작하기에 이 강의는 3점짜리 강의로 판단됩니다.


문장을 입력해주세요 : 교수님과 학생 간에 커뮤니케이션이 거의 없습니다. 과제는 엄청 어렵고 시험도 어려운데 이걸 수업에서 대비시켜 주지 않습니다. 각자 알아서 공부해야해요.
>> 입력하신 내용으로 짐작하기에 이 강의는 3점짜리 강의로 판단됩니다.


문장을 입력해주세요 : 교수님과 학생 간에 커뮤니케이션이 거의 없습니다. 과제는 엄청 어렵고 시험도 어려운데 이걸 수업에서 대비시켜 주지 않습니다. 각자 알아서 공부해야해요. 학점은 짠 편이십니다
>> 입력하신 내용으로 짐작하기에 이 강의는 3점짜리 강의로 판단됩니다.


